In [2]:
import pandas as pd

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

scraping the data using pandas into a list of dataframes

In [4]:
df = pd.read_html(url)

indexing the first element of the list to create our data frame

In [5]:
df_tor = df[0]

In [6]:
df_tor.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


dropping rows where the Borough is not assigned

In [7]:
df_tor.drop(df_tor.loc[df_tor['Borough']=='Not assigned'].index, inplace=True)
df_tor.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


looking for rows where the Neighbourhood is Not assigned but there aren't any once the Borough-less rows were removed

In [8]:
df_tor.loc[df_tor['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


reset the index now that everything is cleaned up

In [9]:
df_tor.reset_index(drop=True,inplace=True)
df_tor.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


showing the shape of the data frame

In [10]:
df_tor.shape

(103, 3)

imported the latitude and longitude data to a new dataframe

In [11]:
url2 = 'https://cocl.us/Geospatial_data'
latlng = pd.read_csv(url2)
latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


set the indeces to postal code in both data frames to make joining easier

In [12]:
df_tor.set_index('Postal Code',inplace=True)
latlng.set_index('Postal Code',inplace=True)

In [13]:
df2 = df_tor.join(latlng)
df2.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


reset the index

In [14]:
df2.reset_index(inplace=True)
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [16]:
import numpy as np
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("libraries imported")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [23]:
from geopy.geocoders import Nominatim

In [24]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent='blue_jay')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('the geographical coordinates of Toronto are {}, {}'.format(latitude,longitude))

the geographical coordinates of Toronto are 43.6534817, -79.3839347


In [35]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df2['Latitude'], df2['Longitude'],df2['Borough'],df2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=False)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto